In [1]:
import pandas as pd

In [2]:
result_file = "test_result_w_succ_diff_nodes_lstm_script_2025_04_07_18_35.csv"

In [3]:
EPS = 1

In [4]:
df = pd.read_csv(result_file)

df = df.rename(columns={"Dataset": "Graph", "Actual": "Rank"})

df.head()

,Graph,Rank,Predicted
0,star_graph_n7,0.0,0.000000
1,star_graph_n7,0.0,0.000000
2,star_graph_n7,3.0,1.698753
3,star_graph_n7,0.0,0.007985
4,star_graph_n7,3.0,2.063117


In [5]:
df['Difference'] = abs(df['Rank']-df['Predicted'])
df['Difference_sq'] = df['Difference']**2
# df.head()

In [6]:
df['EPS'] = df['Rank'].apply(lambda x: EPS if x==0.0 else 0.0)
# df

In [7]:
df['RE'] = df['Difference']/(df['Rank']+df['EPS'])
# df.head()

In [8]:
df

,Graph,Rank,Predicted,Difference,Difference_sq,EPS,RE
0,star_graph_n7,0.0,0.000000,0.000000,0.000000,1.0,0.000000
1,star_graph_n7,0.0,0.000000,0.000000,0.000000,1.0,0.000000
2,star_graph_n7,3.0,1.698753,1.301247,1.693243,0.0,0.433749
3,star_graph_n7,0.0,0.007985,0.007985,0.000064,1.0,0.007985
4,star_graph_n7,3.0,2.063117,0.936883,0.877749,0.0,0.312294
...,...,...,...,...,...,...,...
116046,graph_powerlaw_cluster_graph_n9,2.0,2.214433,0.214433,0.045982,0.0,0.107217
116047,graph_powerlaw_cluster_graph_n9,2.0,1.920420,0.079580,0.006333,0.0,0.039790
116048,graph_powerlaw_cluster_graph_n9,3.0,3.468161,0.468161,0.219175,0.0,0.156054
116049,graph_powerlaw_cluster_graph_n9,3.0,3.101024,0.101024,0.010206,0.0,0.033675


In [9]:
df['RE'].sum()/len(df)

0.06589460638989025

In [10]:
def group_by_table(df, groupby):
    values_count = df[groupby].value_counts().reset_index()
    values_count = values_count.rename(columns={"count": "Count"})
    re_sum = df.groupby(groupby)["RE"].sum().reset_index()
    mre_error = df.groupby(groupby)["RE"].mean().reset_index()
    mre_error = mre_error.rename(columns={"RE": "MRE"})
    mse_error = df.groupby(groupby)["Difference_sq"].mean().reset_index()
    mse_error = mse_error.rename(columns={"Difference_sq": "MSE"})

    return values_count, re_sum, mre_error, mse_error

# Group by Rank

In [11]:
groupby = ['Rank']

values_count, re_sum, mre_error, mse_error = group_by_table(df, groupby)

df_result = pd.merge(values_count, re_sum).sort_values(by=['Count', 'RE'], ascending=[False, False])
df_result = pd.merge(df_result, mre_error)
df_result = pd.merge(df_result, mse_error)
df_result.to_csv("analysis_results/test_analysis_grp_by_rank.csv", index=False)
df_result


,Rank,Count,RE,MRE,MSE
0,3.0,28122,2225.381679,0.079133,0.099115
1,2.0,27607,2896.837003,0.104931,0.072553
2,0.0,17578,32.790340,0.001865,0.000036
3,4.0,17567,1016.103971,0.057842,0.091488
4,1.0,16573,972.443512,0.058676,0.006221
5,5.0,6078,326.150193,0.053661,0.112976
6,6.0,1282,75.862680,0.059175,0.183916
7,7.0,506,19.715143,0.038963,0.114658
8,8.0,365,23.414832,0.064150,0.298087
9,9.0,204,24.751170,0.121329,1.236879


# Group by Dataset

In [12]:
groupby = ['Graph']

values_count, re_sum, mre_error, mse_error = group_by_table(df, groupby)

df_result = pd.merge(values_count, re_sum).sort_values(by=['Count', 'RE'], ascending=[False, False])
df_result = pd.merge(df_result, mre_error)
df_result = pd.merge(df_result, mse_error)
df_result.to_csv("analysis_results/test_analysis_grp_by_dataset.csv", index=False)
df_result


,Graph,Count,RE,MRE,MSE
0,graph_powerlaw_cluster_graph_n9,69120,4713.137740,0.068188,0.061189
1,graph_random_regular_graph_n8_d4,19532,1648.766203,0.084414,0.093146
2,star_graph_n15,12288,171.398181,0.013948,0.115249
3,graph_powerlaw_cluster_graph_n8,7168,628.727628,0.087713,0.078338
4,graph_random_regular_graph_n7_d4,3907,320.300627,0.081981,0.067773
5,star_graph_n13,2663,56.124355,0.021076,0.120134
6,graph_powerlaw_cluster_graph_n7,1350,105.743817,0.078329,0.051415
7,star_graph_n7,23,2.936414,0.127670,0.616616


# Group by Dataset W/O Rank 0

In [13]:
groupby = ['Graph']

df_wo_0 = df[df['Rank']!=0]
values_count, re_sum, mre_error, mse_error = group_by_table(df_wo_0, groupby)

df_result = pd.merge(values_count, re_sum).sort_values(by=['Count', 'RE'], ascending=[False, False])
df_result = pd.merge(df_result, mre_error)
df_result = pd.merge(df_result, mse_error)
df_result.to_csv("analysis_results/test_analysis_grp_by_dataset_wo_rank0.csv", index=False)
df_result

,Graph,Count,RE,MRE,MSE
0,graph_powerlaw_cluster_graph_n9,65799,4693.296268,0.071328,0.064271
1,graph_random_regular_graph_n8_d4,19180,1646.637934,0.085852,0.094853
2,graph_powerlaw_cluster_graph_n8,6403,623.277843,0.097342,0.087678
3,graph_random_regular_graph_n7_d4,3764,319.465619,0.084874,0.070342
4,star_graph_n15,1617,168.321499,0.104095,0.875800
5,graph_powerlaw_cluster_graph_n7,1252,105.194702,0.084021,0.055429
6,star_graph_n13,450,55.230316,0.122734,0.710916
7,star_graph_n7,8,2.920445,0.365056,1.772754


# Group by Dataset and Rank

In [14]:
groupby = ['Graph', 'Rank']

values_count, re_sum, mre_error, mse_error = group_by_table(df, groupby)

df_result = pd.merge(values_count, re_sum).sort_values(by=['Graph', 'Count'], ascending=[True, False])
df_result = pd.merge(df_result, mre_error)
df_result = pd.merge(df_result, mse_error)
df_result.to_csv("analysis_results/test_analysis_grp_by_dataset_rank.csv", index=False)
df_result

,Graph,Rank,Count,RE,MRE,MSE
0,graph_powerlaw_cluster_graph_n7,2.0,414,43.911066,0.106065,0.069718
1,graph_powerlaw_cluster_graph_n7,1.0,388,32.179854,0.082938,0.012408
2,graph_powerlaw_cluster_graph_n7,3.0,323,22.527802,0.069746,0.069274
3,graph_powerlaw_cluster_graph_n7,4.0,119,5.606689,0.047115,0.061435
4,graph_powerlaw_cluster_graph_n7,0.0,98,0.549115,0.005603,0.000142
...,...,...,...,...,...,...
58,star_graph_n7,0.0,15,0.015969,0.001065,0.000009
59,star_graph_n7,3.0,3,1.164823,0.388274,1.383126
60,star_graph_n7,2.0,2,0.741145,0.370573,0.727392
61,star_graph_n7,4.0,2,0.523679,0.261839,1.277893
